# Nội suy ngược

<b>Bài toán:</b>
Tìm x biết $f(x)=y$ trong đó, hàm $f(x)$ chỉ có dạng bảng

<b>Ý tưởng:</b>

<ol>
<li>Xác định khoảng cách ly:
<ul>
    <li> Xác định các khoảng đơn điệu</li>
    <li> Xác định khoảng đơn điệu có chứa nghiệm</li>
</ul></li>
<li>Tìm nghiệm trên khoảng đơn điệu có chứa nghiệm hoặc khoảng <i>không</i> đơn điệu nhưng có nghiệm duy nhất</li>
</ol>

<b>Hai phương pháp tìm nghiệm</b>

PP1: Xây dựng xấp xỉ của hàm ngược \
    Hàm đơn điệu $[a,b] \rightarrow Im([a,b])$ là hàm song ánh => tồn tại ánh xạ ngược $Im([a,b]) \rightarrow [a,b]: x=g(y)$ \
    Điều kiện mốc nội suy $x_i \neq x_j \forall i \neq j \Rightarrow y_i \neq y_j \forall i \neq j$ \
    Xem y là ẩn, $x=g(y) \approx P_n(y)$ là hàm 

PP2: Phương pháp lặp để tìm nghiệm
<ul>
<li>Chọn 2 mốc liên tiếp chặn trên, chặn dưới của nghiệm</li>
<li>Nếu chọn mốc chặn dưới, xây dựng công thức lặp từ đa thức nội suy Newton tiến. Nếu chọn mốc chặn trên thì Newton lùi.(Chọn bên nào có ít mốc hơn)</li>
</ul>

Xét trường hợp chọn mốc bên trái, ta có:
$$P_n(x_k+th)=y_k+\Delta y_kt + \frac{\Delta^2y_k}{2!}t(t-1)+... = \bar y $$
Khi đó,
$$\begin{cases}
t &=& \frac{1}{\Delta y_k}[\bar y - y_k - (\frac{\Delta^2y_k}{2!}t(t-1)+...)] = \phi(t) \\
t_0 &=& \frac{\bar y - y_k}{\Delta y_k}, \hspace{.2cm} t_{i+1} = \phi(t_i)
\end{cases}$$
Sai số,
$$\frac{|t_{i+1}-t_i|}{|t_{i+1}|} < \delta$$

### Import packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt

### Input chương trình

In [2]:
def input_data():
    x_t: list[float] = []
    y_t: list[float] = []
    with open('input.txt', 'r+') as f:
        for line in f.readlines():
            x_temp = float(line.split(' ')[0])
            y_temp = float(line.split(' ')[1])
            if x_temp in x_t:
                continue
            x_t.append(x_temp)
            y_t.append(y_temp)
    f.close()
    return x_t, y_t

In [4]:
def is_equidistant(vec):
    h = np.diff(vec)
    for i in range(len(h)-1):
        if abs(h[i] - h[i+1]) >= 1e-7:
            return False
    return True